In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Load Color Image Augmented Dataset 

In [ ]:
def load_train():
  data = []
  labels = []

# grab the image paths and randomly shuffle them
  print("[INFO] loading images...")
  imagePaths_list = []
  imagePaths = "/content/drive/MyDrive/Wilson/Augmented Dataset by Histo Score - 24 hrs/Histo Score 1-2"
  #print(os.listdir(imagePaths))
  counter1 = 0
  for fileName in os.listdir(imagePaths):
	  imagePaths_list.append(os.path.join(imagePaths, fileName))
	  #counter1 = counter1 + 1
	#if counter1 == 500:
		#break
  imagePaths = "/content/drive/MyDrive/Wilson/Augmented Dataset by Histo Score - 24 hrs/Histo Score 3-4"
  counter2 = 0
  for fileName in os.listdir(imagePaths):
	  imagePaths_list.append(os.path.join(imagePaths, fileName))
	  counter2 = counter2 + 1
	#if counter2 == 500:
		#break
  print(len(imagePaths_list))
  print(imagePaths_list[0])
  #print(imagePaths_list)
  random.seed(42)
  random.shuffle(imagePaths_list)

# loop over the input images
  counter = 1
  for imagePath in imagePaths_list:
	# load the image, pre-process it, and store it in the data list
	#print(imagePath)
	#var_name = imagePath.split(os.path.sep)[-1]
	#print(counter)
	#print(var_name)
	  image = cv2.imread(imagePath)#scipy.io.loadmat(imagePath)[var_name[:-4]]
	#image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
	  image = img_to_array(image)
	  data.append(image)
 
	# extract the class label from the image path and update the
	# labels list
	  label = imagePath.split(os.path.sep)[-2]
	  labels.append(label)
	  counter = counter + 1
# scale the raw pixel intensities to the range [0, 1]
  data = np.array(data, dtype="float")
#print(labels)
  labels = np.array(labels)
#print("[INFO] data matrix: {:.2f}MB".format(
	#data.nbytes / (1024 * 1000.0)))

# binarize the labels
  lb = LabelBinarizer()
  labels = lb.fit_transform(labels)
  return data, labels
# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing


Load Color Image Validation Set

In [ ]:
def load_val():
  imagePaths = "/content/drive/MyDrive/Wilson/Raw Data Set by Histo Score - 24 hrs/Histo Score 1-2"
  imagePaths_list_test = []
  for fileName in os.listdir(imagePaths):
    imagePaths_list_test.append(os.path.join(imagePaths, fileName))

  imagePaths = "/content/drive/MyDrive/Wilson/Raw Data Set by Histo Score - 24 hrs/Histo Score 3-4"
  for fileName in os.listdir(imagePaths):
    imagePaths_list_test.append(os.path.join(imagePaths, fileName))
  test_data = []
  test_labels = []
  print("length of images_paths_test " + str(imagePaths_list_test))
  random.seed(42)
  random.shuffle(imagePaths_list_test)

  for imagePath in imagePaths_list_test:
	# load the image, pre-process it, and store it in the data list
	#print(imagePath)
	#var_name = imagePath.split(os.path.sep)[-1]
    image = cv2.imread(imagePath)#[var_name[:-4]]
    image = cv2.resize(image, (96, 96))
    image = img_to_array(image)
    test_data.append(image)
 
	# extract the class label from the image path and update the
	# labels list
    label = imagePath.split(os.path.sep)[-2]
    test_labels.append(label)
  test_data = np.array(test_data, dtype="float")/255.0
#print(labels)
  test_labels = np.array(test_labels)
#print("[INFO] data matrix: {:.2f}MB".format(
	#data.nbytes / (1024 * 1000.0)))

# binarize the labels
  lb = LabelBinarizer()
  test_labels = lb.fit_transform(test_labels)
  return test_data, test_labels

Data Augmentation

In [ ]:
import matplotlib
matplotlib.use("Agg")
import scipy.io
import glob
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
#from pyimagesearch.smallervggnet import SmallerVGGNet
from keras.models import Sequential
from keras.layers import Dense, Conv2D, BatchNormalization, MaxPooling2D, Activation, Flatten, Dropout
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import argparse
import random
import pickle
import cv2
import os
#from tk import *
from sklearn.utils import compute_class_weight


# initialize the data and labels
data_cat1 = []
labels_cat1 = []
data_cat2 = []
labels_cat2 = []

# grab the image paths and randomly shuffle them
print("[INFO] loading images...")
imagePaths_list_cat1 = []
imagePaths = "/content/drive/MyDrive/Wilson/Raw Data Set by Histo Score - 24 hrs/Histo Score 1-2"
for fileName in os.listdir(imagePaths):
    #print(imagePaths + "/" + str(fileName) + " " + str(fileName[:-4]))
    image = cv2.imread(imagePaths + "/" + str(fileName))
    image = cv2.resize(image, (96, 96))
    image = img_to_array(image)
    print(image.shape)
    data_cat1.append(image)
 
	  # extract the class label from the image path and update the
	  # labels list
    label = "Histo Score 1-2"
    labels_cat1.append(label)
imagePaths2 = "/content/drive/MyDrive/Wilson/Raw Data Set by Histo Score - 24 hrs/Histo Score 3-4"
imagePaths_list_cat2 = []
for fileName in os.listdir(imagePaths2):
    #print(imagePaths2 + "/" + str(fileName) + " " + str(fileName[:-4]))
    image = cv2.imread(imagePaths2 + "/" + str(fileName))
    image = cv2.resize(image, (96, 96))
    image = img_to_array(image)
    data_cat2.append(image)

    label = "Histo Score 3-4"
    labels_cat2.append(label)
# loop over the input images
#for imagePath in imagePaths_list_cat1:
	## load the image, pre-process it, and store it in the data list
	##print(imagePath)
	#var_name = imagePath.split(os.path.sep)[-1]
	#image = scipy.io.loadmat(imagePath)[var_name[:-4]]
	#image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
	#image = np.array(image)
	#data_cat1.append(image)
 
	# extract the class label from the image path and update the
	# labels list
	#label = imagePath.split(os.path.sep)[-2]
	#labels_cat1.append(label)

# scale the raw pixel intensities to the range [0, 1]

data_cat1 = np.array(data_cat1, dtype = "float")/255.0 
#print(labels)
labels_cat1 = np.array(labels_cat1)
print("[INFO] data matrix: {:.2f}MB".format(
	data_cat1.nbytes / (1024 * 1000.0)))

# binarize the labels
lb = LabelBinarizer()
labels_cat1 = lb.fit_transform(labels_cat1)

#for imagePath in imagePaths_list_cat2:
	# load the image, pre-process it, and store it in the data list
	#print(imagePath)
	#var_name = imagePath.split(os.path.sep)[-1]
	#image = scipy.io.loadmat(imagePath)[var_name[:-4]]
	#image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
	#image = np.array(image)
	#data_cat2.append(image)
 
	# extract the class label from the image path and update the
	# labels list
	#label = imagePath.split(os.path.sep)[-2]
	#labels_cat2.append(label)

# scale the raw pixel intensities to the range [0, 1]
data_cat2 = np.array(data_cat2, dtype = "float")/255.0 
#print(labels)
labels_cat2 = np.array(labels_cat2)
print("[INFO] data matrix: {:.2f}MB".format(
	data_cat2.nbytes / (1024 * 1000.0)))

# binarize the labels
lb = LabelBinarizer()
labels_cat2 = lb.fit_transform(labels_cat2)


aug = ImageDataGenerator(rotation_range=30,zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15,horizontal_flip=True,fill_mode="nearest")
imageGen_cat1 = aug.flow(data_cat1)
i = 1
#CATEGORY 1-2
for image_list in imageGen_cat1:
    print(image_list.shape)
    for image in image_list:
        mdic = {"Generated Image " + str(i): image}
        print(image.size)
        plt.imsave("/content/drive/MyDrive/Wilson/Augmented Dataset by Histo Score - 24 hrs/Histo Score 1-2/Generated Image " + str(i) + ".png", image)
        #print(i)
        if i == 2000:
            break
        i = i+1
    else:
        continue
    break


aug = ImageDataGenerator(rotation_range=30,zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15,horizontal_flip=True,fill_mode="nearest")
imageGen_cat2 = aug.flow(data_cat2)
i = 2001
for image_list in imageGen_cat2:
    print(image_list.shape)
    for image in image_list:
        mdic = {"Generated Image " + str(i): image}
        #print(mdic)
        plt.imsave("/content/drive/MyDrive/Wilson/Augmented Dataset by Histo Score - 24 hrs/Histo Score 3-4/Generated Image " + str(i) + ".png", image)
        if i == 4000:
            break
        i = i+1
    else:
        continue
    break

[INFO] loading images...
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)


KeyboardInterrupt: ignored

In [ ]:
from sys import breakpointhook
#Normal Neural Net Architecture, Train Test split, image augmentation, Class Weightage, Test-Time Augmentation

# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
import scipy.io
from tensorflow.keras.utils import img_to_array

import glob
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
#from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
#from pyimagesearch.smallervggnet import SmallerVGGNet
from keras.models import Sequential
from keras.layers import Dense, Conv2D, BatchNormalization, MaxPooling2D, Activation, Flatten, Dropout
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import argparse
import random
import pickle
import cv2
import os
#from tk import *
from sklearn.utils import compute_class_weight


# construct the argument parse and parse the arguments
#ap = argparse.ArgumentParser()
#ap.add_argument("-d", "--dataset", required=True,help="path to input dataset (i.e., directory of images)")
#ap.add_argument("-m", "--model", required=True,help="path to output model")
#ap.add_argument("-l", "--labelbin", required=True,help="path to output label binarizer")
#ap.add_argument("-p", "--plot", type=str, default="plot.png",help="path to output accuracy/loss plot")
#args = vars(ap.parse_args())

# initialize the number of epochs to train for, initial learning rate,
# batch size, and image dimensions
EPOCHS = 100
INIT_LR = 0.01
BS = 80
IMAGE_DIMS = (96, 96, 3)

# initialize the data and labels
data = []
labels = []

# grab the image paths and randomly shuffle them
print("[INFO] loading images...")
imagePaths_list = []
imagePaths = "/content/drive/MyDrive/Wilson/Augmented Dataset by Histo Score - 24 hrs/Histo Score 1-2"
#print(os.listdir(imagePaths))
counter1 = 0
for fileName in os.listdir(imagePaths):
	imagePaths_list.append(os.path.join(imagePaths, fileName))
	counter1 = counter1 + 1
	#if counter1 == 500:
		#break
imagePaths = "/content/drive/MyDrive/Wilson/Augmented Dataset by Histo Score - 24 hrs/Histo Score 3-4"
counter2 = 0
for fileName in os.listdir(imagePaths):
	imagePaths_list.append(os.path.join(imagePaths, fileName))
	counter2 = counter2 + 1
	#if counter2 == 500:
		#break
print(len(imagePaths_list))
print(imagePaths_list[0])
#print(imagePaths_list)
random.seed(42)
random.shuffle(imagePaths_list)

# loop over the input images
counter = 1
for imagePath in imagePaths_list:
	# load the image, pre-process it, and store it in the data list
	#print(imagePath)
	#var_name = imagePath.split(os.path.sep)[-1]
	#print(counter)
	#print(var_name)
	image = cv2.imread(imagePath)#scipy.io.loadmat(imagePath)[var_name[:-4]]
	#image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
	image = img_to_array(image)
	data.append(image)
 
	# extract the class label from the image path and update the
	# labels list
	label = imagePath.split(os.path.sep)[-2]
	labels.append(label)
	counter = counter + 1
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float")
#print(labels)
labels = np.array(labels)
#print("[INFO] data matrix: {:.2f}MB".format(
	#data.nbytes / (1024 * 1000.0)))

# binarize the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
(trainX, testX, trainY, testY) = train_test_split(data,
	labels, test_size=0.2, random_state=42)

#(trainX, testX, trainY, testY) = train_test_split(training_data,
#	training_labels, test_size=0.2, random_state=42)

# initialize the model
print("[INFO] compiling model...")
#model = SmallerVGGNet.build(width=IMAGE_DIMS[1], height=IMAGE_DIMS[0], depth=IMAGE_DIMS[2], classes=len(lb.classes_))
model = Sequential() 
#Conv layer: 16 3x3 kernels with (211,211,16 output)
#ReLu layer
#batch normalization 
#conv layer: 16 3x3 kernels with (211, 211, 16 output)
#ReLU layer
#batch normalization
#pooling: 2x2 pool size with 1x1 stride: output shape (211, 211, 16)
#dropout layer
#FC: 32
#ReLU
#batch normalization
#dropout
#FC: 2
#softmax: 2

#NOTE: repeat again if needed; the above architecture is shallow 
model.add(Conv2D(16, (3, 3), padding="same", input_shape=(96, 96, 3)))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(16, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(32, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation("sigmoid"))

from tensorflow.keras.optimizers import Adam
# Load the raw images as test
imagePaths = "/content/drive/MyDrive/Wilson/Raw Data Set by Histo Score - 24 hrs/Histo Score 1-2"
imagePaths_list_test = []
for fileName in os.listdir(imagePaths):
	imagePaths_list_test.append(os.path.join(imagePaths, fileName))

imagePaths = "/content/drive/MyDrive/Wilson/Raw Data Set by Histo Score - 24 hrs/Histo Score 3-4"
for fileName in os.listdir(imagePaths):
	imagePaths_list_test.append(os.path.join(imagePaths, fileName))
test_data = []
test_labels = []
print("length of images_paths_test " + str(imagePaths_list_test))
random.seed(42)
random.shuffle(imagePaths_list_test)

for imagePath in imagePaths_list_test:
	# load the image, pre-process it, and store it in the data list
	#print(imagePath)
	#var_name = imagePath.split(os.path.sep)[-1]
	image = cv2.imread(imagePath)#[var_name[:-4]]
	image = cv2.resize(image, (96, 96))
	image = img_to_array(image)
	test_data.append(image)
 
	# extract the class label from the image path and update the
	# labels list
	label = imagePath.split(os.path.sep)[-2]
	test_labels.append(label)
test_data = np.array(test_data, dtype="float")/255.0 
#print(labels)
test_labels = np.array(test_labels)
#print("[INFO] data matrix: {:.2f}MB".format(
	#data.nbytes / (1024 * 1000.0)))

# binarize the labels
lb = LabelBinarizer()
test_labels = lb.fit_transform(test_labels)

aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
	height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
	horizontal_flip=True, fill_mode="nearest")
opt = Adam(lr=0.01)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])
model.summary()
print("[INFO] training network...")
H = model.fit(
	trainX, trainY, batch_size = BS,
	validation_data = (testX, testY),
	steps_per_epoch=(len(trainX)//BS),
	epochs=EPOCHS, verbose=1) #class_weight=classWeight)

# save the model to disk
print("[INFO] serializing network...")
model.save("burnclassifiersfdi.model")

# save the label binarizer to disk
print("[INFO] serializing label binarizer...")
f = open("lbsfdi.pickle", "wb")
f.write(pickle.dumps(lb))
f.close()

[INFO] loading images...
4000
/content/drive/MyDrive/Wilson/Augmented Dataset by Histo Score - 24 hrs/Histo Score 1-2/Generated Image 1001.png
[INFO] compiling model...
length of images_paths_test ['/content/drive/MyDrive/Wilson/Raw Data Set by Histo Score - 24 hrs/Histo Score 1-2/P5-3.png', '/content/drive/MyDrive/Wilson/Raw Data Set by Histo Score - 24 hrs/Histo Score 1-2/P5-5.png', '/content/drive/MyDrive/Wilson/Raw Data Set by Histo Score - 24 hrs/Histo Score 1-2/P5-7.png', '/content/drive/MyDrive/Wilson/Raw Data Set by Histo Score - 24 hrs/Histo Score 1-2/P5-10.png', '/content/drive/MyDrive/Wilson/Raw Data Set by Histo Score - 24 hrs/Histo Score 1-2/P5-12.png', '/content/drive/MyDrive/Wilson/Raw Data Set by Histo Score - 24 hrs/Histo Score 1-2/P5-16.png', '/content/drive/MyDrive/Wilson/Raw Data Set by Histo Score - 24 hrs/Histo Score 1-2/P6-2.png', '/content/drive/MyDrive/Wilson/Raw Data Set by Histo Score - 24 hrs/Histo Score 1-2/P6-4.png', '/content/drive/MyDrive/Wilson/Raw Data

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


[INFO] training network...
Epoch 1/100
40/40 [==============================] - 2s 26ms/step - loss: 0.6684 - accuracy: 0.6609 - val_loss: 98.4457 - val_accuracy: 0.4913
Epoch 2/100
40/40 [==============================] - 1s 16ms/step - loss: 0.4509 - accuracy: 0.7844 - val_loss: 113.3998 - val_accuracy: 0.4913
Epoch 3/100
40/40 [==============================] - 1s 16ms/step - loss: 0.4212 - accuracy: 0.7962 - val_loss: 29.6017 - val_accuracy: 0.4913
Epoch 4/100
40/40 [==============================] - 1s 16ms/step - loss: 0.4443 - accuracy: 0.7856 - val_loss: 2.6957 - val_accuracy: 0.4913
Epoch 5/100
40/40 [==============================] - 1s 16ms/step - loss: 0.3647 - accuracy: 0.8322 - val_loss: 2.9293 - val_accuracy: 0.5763
Epoch 6/100
40/40 [==============================] - 1s 16ms/step - loss: 0.3118 - accuracy: 0.8609 - val_loss: 16.9987 - val_accuracy: 0.5138
Epoch 7/100
40/40 [==============================] - 1s 16ms/step - loss: 0.3176 - accuracy: 0.8597 - val_loss: 21.5

[INFO] serializing label binarizer...


In [ ]:
results = model.evaluate(test_data, test_labels)
print(results)

3/3 [==============================] - 0s 19ms/step - loss: 208.3932 - accuracy: 0.5750
[208.39321899414062, 0.574999988079071]


In [ ]:
!pip install scikeras
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense, Conv2D, BatchNormalization, MaxPooling2D, Activation, Flatten, Dropout
import numpy as np
def create_model():
	# create model
	model = Sequential() 
	model.add(Conv2D(16, (3, 3), padding="same", input_shape=IMAGE_DIMS))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=-1))
	model.add(Conv2D(16, (3, 3), padding="same"))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=-1))
	model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
	model.add(Dropout(0.25))

	model.add(Conv2D(32, (3, 3), padding="same"))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=-1))
	model.add(Conv2D(32, (3, 3), padding="same"))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=-1))
	model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
	model.add(Dropout(0.25))

	model.add(Flatten())
	model.add(Dense(128))
	model.add(Activation("relu"))
	model.add(BatchNormalization())
	model.add(Dropout(0.5))
	model.add(Dense(1))
	model.add(Activation("sigmoid"))

	# Compile model
	model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])
	return model
seed = 7
np.random.seed(seed)
# create model
model = KerasClassifier(model=create_model, epochs=100, batch_size=80)
# evaluate using 10-fold cross validation
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(model, data, labels, cv=kfold, scoring= 'accuracy')
print(results)
print(results.keys())

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Epoch 1/100
45/45 [==============================] - 2s 35ms/step - loss: 0.7796 - accuracy: 0.6103
Epoch 2/100
45/45 [==============================] - 2s 33ms/step - loss: 0.5903 - accuracy: 0.6589
Epoch 3/100
45/45 [==============================] - 1s 33ms/step - loss: 0.5376 - accuracy: 0.7125
Epoch 4/100
45/45 [==============================] - 2s 33ms/step - loss: 0.4910 - accuracy: 0.7547
Epoch 5/100
45/45 [==============================] - 2s 33ms/step - loss: 0.4405 - accuracy: 0.7878
Epoch 6/100
45/45 [==============================] - 2s 34ms/step - loss: 0.4012 - accuracy: 0.8086
Epoch 7/100
45/45 [==============================] - 1s 33ms/step - loss: 0.3807 - accuracy: 0.8231
Epoch 8/100
45/45 [==============================] - 2s 33ms/step - loss: 0.3557 - accuracy: 0.8400
Epoch 9/100
45/45 [==============================] - 2s 39ms/step - loss: 0.3332 - accuracy: 0.8506
E

AttributeError: ignored

TRANSFER LEARNING

In [ ]:
from tensorflow.keras.applications import ResNet50V2
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from keras.layers import Dense, Conv2D, BatchNormalization, MaxPooling2D, Activation, Flatten, Dropout, Input
import matplotlib
matplotlib.use("Agg")
import scipy.io
import glob
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
#from pyimagesearch.smallervggnet import SmallerVGGNet
from keras.models import Sequential
from keras.layers import Dense, Conv2D, BatchNormalization, MaxPooling2D, Activation, Flatten, Dropout
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import argparse
import random
import pickle
import cv2
import os
#from tk import *
from sklearn.utils import compute_class_weight
from sklearn.model_selection import StratifiedKFold
EPOCHS = 100
INIT_LR = 0.01
BS = 80
IMAGE_DIMS = (96, 96, 3)
def multify_weights(kernel, out_channels):
  mean_1d = np.mean(kernel, axis=-2).reshape(kernel[:,:,-1:,:].shape)
  tiled = np.tile(mean_1d, (out_channels, 1))
  return(tiled)


# Loop through layers of both original model 
# and custom model and copy over weights 
# layer_modify refers to first convolutional layer
def copy_weights_tl(model_orig, custom_model, layer_modify):
  layer_to_modify = [layer_modify]

  conf = custom_model.get_config()
  layer_names = [conf['layers'][x]['name'] for x in range(len(conf['layers']))]

  for layer in model_orig.layers:
    if layer.name in layer_names:
      if layer.get_weights() != []:
        target_layer = custom_model.get_layer(layer.name)

        if layer.name in layer_to_modify:    
          kernels = layer.get_weights()[0]
          biases  = layer.get_weights()[1]

          kernels_extra_channel = np.concatenate((kernels,
                                                  multify_weights(kernels, 8 - 3)),
                                                  axis=-2)
                                                  
          target_layer.set_weights([kernels_extra_channel, biases])
          target_layer.trainable = False

        else:
          target_layer.set_weights(layer.get_weights())
          target_layer.trainable = False
def semantic_segmentation(input_size, img_height, img_width, input_channel, pretrained_weights=None):
    inputs = Input(input_size)

    resnet50 = ResNet50V2(weights='imagenet', include_top=False)

    config = resnet50.get_config()
    config["layers"][0]["config"]["batch_input_shape"] = (None, img_height, img_width, input_channel)
    config["layers"][2]["config"]["strides"] = (1, 1)


    #resnet_custom = tf.keras.models.Model.from_config(config)
    #modify_name = config["layers"][2]["config"]["name"]


    #copy_weights_tl(resnet50, resnet_custom, modify_name)
    conv4 = resnet50(inputs)
    conv4 = tf.keras.activations.relu(conv4)
    #conv4_up = UpSampling2D(size=(16, 16), interpolation='bilinear')(conv4)

    #red = GlobalAveragePooling2D(name='red_pool')(conv4)
    #red = tf.keras.layers.Reshape((1, 1, 2048))(red)
    #red = Conv2D(128, 1, name='red_1_by_1')(red)
    #red = UpSampling2D(size=(320, 320), interpolation='bilinear', name='red_upsampling')(red)

    #yellow = AveragePooling2D(pool_size=(2,2), name='yellow_pool')(conv4)
    #yellow = Conv2D(128, 1, name='yellow_1_by_1')(yellow)
    #yellow = UpSampling2D(size=(32, 32), interpolation='bilinear', name='yellow_upsampling')(yellow)

    #blue = AveragePooling2D(pool_size=(4,4), name='blue_pool', padding="valid")(conv4)
    #blue = Conv2D(128, 1, name='blue_1_by_1')(blue)
    #blue = UpSampling2D(size=(16*4, 16*4), interpolation='bilinear', name='blue_upsampling')(blue)

    #green = AveragePooling2D(pool_size=(8,8), name='green_pool', padding="valid")(conv4)
    #green = Conv2D(128, 1, name='green_1_by_2')(green)
    #green = UpSampling2D(size=(160, 160), interpolation='bilinear', name='green_upsampling')(green)

    #merge9 = concatenate([conv4_up, red, yellow, blue, green])

    #merge9 = Conv2D(128, 3, activation="relu", padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv4 = Flatten()(conv4)
    conv4 = Dense(128)(conv4)
    conv4 = Activation("relu")(conv4)
    conv4 = BatchNormalization()(conv4)
    conv4 = Dropout(0.5)(conv4)
    conv4 = Dense(1)(conv4)
    conv4 = Activation("sigmoid")(conv4)

    #output = Conv2D(3, 1, activation = 'softmax', padding='same')(merge9)
    

    model = Model(inputs, conv4)
    opt = SGD(lr=0.01)
    model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])
    print(model.summary())
    return(model)
model_transfer = semantic_segmentation(input_size=(96,96,3), img_height = 96, img_width = 96, input_channel = 3)
print("[INFO] training network...")
data, labels = load_train()
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.2, random_state=42)
H = model_transfer.fit(
	trainX, trainY, batch_size = BS,
	validation_data = (testX, testY),
	steps_per_epoch=(len(trainX)//BS),
	epochs=EPOCHS, verbose=1) #class_weight=classWeight)

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 resnet50v2 (Functional)     (None, None, None, 2048)  23564800  
                                                                 
 tf.nn.relu_2 (TFOpLambda)   (None, 3, 3, 2048)        0         
                                                                 
 flatten_2 (Flatten)         (None, 18432)             0         
                                                                 
 dense_4 (Dense)             (None, 128)               2359424   
                                                                 
 activation_4 (Activation)   (None, 128)               0         
                                                                 
 batch_normalization_2 (Batc  (None, 128)              512 

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


None
[INFO] training network...
[INFO] loading images...
4000
/content/drive/MyDrive/Wilson/Augmented Dataset by Histo Score - 24 hrs/Histo Score 1-2/Generated Image 1001.png
Epoch 1/100
40/40 [==============================] - 7s 65ms/step - loss: 0.4880 - accuracy: 0.7850 - val_loss: 1.2880 - val_accuracy: 0.5300
Epoch 2/100
40/40 [==============================] - 2s 39ms/step - loss: 0.0908 - accuracy: 0.9675 - val_loss: 1.1007 - val_accuracy: 0.5275
Epoch 3/100
40/40 [==============================] - 2s 39ms/step - loss: 0.0214 - accuracy: 0.9966 - val_loss: 0.8354 - val_accuracy: 0.6612
Epoch 4/100
40/40 [==============================] - 2s 39ms/step - loss: 0.0127 - accuracy: 0.9975 - val_loss: 0.6326 - val_accuracy: 0.7875
Epoch 5/100
40/40 [==============================] - 2s 39ms/step - loss: 0.0074 - accuracy: 1.0000 - val_loss: 0.3676 - val_accuracy: 0.8863
Epoch 6/100
40/40 [==============================] - 2s 40ms/step - loss: 0.0060 - accuracy: 0.9997 - val_loss: 0.1

In [ ]:
test_data, test_labels = load_val()
results = model_transfer.evaluate(test_data, test_labels)
print(results)

length of images_paths_test ['/content/drive/MyDrive/Wilson/Raw Data Set by Histo Score - 24 hrs/Histo Score 1-2/P5-3.png', '/content/drive/MyDrive/Wilson/Raw Data Set by Histo Score - 24 hrs/Histo Score 1-2/P5-5.png', '/content/drive/MyDrive/Wilson/Raw Data Set by Histo Score - 24 hrs/Histo Score 1-2/P5-7.png', '/content/drive/MyDrive/Wilson/Raw Data Set by Histo Score - 24 hrs/Histo Score 1-2/P5-10.png', '/content/drive/MyDrive/Wilson/Raw Data Set by Histo Score - 24 hrs/Histo Score 1-2/P5-12.png', '/content/drive/MyDrive/Wilson/Raw Data Set by Histo Score - 24 hrs/Histo Score 1-2/P5-16.png', '/content/drive/MyDrive/Wilson/Raw Data Set by Histo Score - 24 hrs/Histo Score 1-2/P6-2.png', '/content/drive/MyDrive/Wilson/Raw Data Set by Histo Score - 24 hrs/Histo Score 1-2/P6-4.png', '/content/drive/MyDrive/Wilson/Raw Data Set by Histo Score - 24 hrs/Histo Score 1-2/P6-6.png', '/content/drive/MyDrive/Wilson/Raw Data Set by Histo Score - 24 hrs/Histo Score 1-2/P6-10.png', '/content/drive/M

K-Fold Cross Validation:


In [ ]:
#model_transfer_kfold = semantic_segmentation(input_size=(211,211,8), img_height = 211, img_width = 211, input_channel = 8)
!pip install scikeras
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv2D, BatchNormalization, MaxPooling2D, Activation, Flatten, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50V2
import tensorflow as tf
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import KFold
import os
import random
import cv2
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.applications import ResNet50V2
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from keras.layers import Dense, Conv2D, BatchNormalization, MaxPooling2D, Activation, Flatten, Dropout, Input
import matplotlib
matplotlib.use("Agg")
import scipy.io
import glob
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
#from pyimagesearch.smallervggnet import SmallerVGGNet
from keras.models import Sequential
from keras.layers import Dense, Conv2D, BatchNormalization, MaxPooling2D, Activation, Flatten, Dropout
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import argparse
import random
import pickle
import cv2
import os
#from tk import *
from sklearn.utils import compute_class_weight
from sklearn.model_selection import StratifiedKFold
seed = 7
np.random.seed(seed)
def semantic_segmentation(input_size, img_height, img_width, input_channel, pretrained_weights=None):
    inputs = Input(input_size)

    resnet50 = ResNet50V2(weights='imagenet', include_top=False)

    config = resnet50.get_config()
    config["layers"][0]["config"]["batch_input_shape"] = (None, img_height, img_width, input_channel)
    config["layers"][2]["config"]["strides"] = (1, 1)


    #resnet_custom = tf.keras.models.Model.from_config(config)
    #modify_name = config["layers"][2]["config"]["name"]


    #copy_weights_tl(resnet50, resnet_custom, modify_name)
    conv4 = resnet50(inputs)
    conv4 = tf.keras.activations.relu(conv4)
    conv4 = Flatten()(conv4)
    conv4 = Dense(128)(conv4)
    conv4 = Activation("relu")(conv4)
    conv4 = BatchNormalization()(conv4)
    conv4 = Dropout(0.5)(conv4)
    conv4 = Dense(1)(conv4)
    conv4 = Activation("sigmoid")(conv4)

    #output = Conv2D(3, 1, activation = 'softmax', padding='same')(merge9)
    

    model = Model(inputs, conv4)
    opt = SGD(lr=0.01)
    model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])
    print(model.summary())
    return(model)
# create model
model_transfer_kfold = KerasClassifier(model=semantic_segmentation(input_size=(96,96,3), img_height = 96, img_width = 96, input_channel = 3), epochs=30, batch_size=80)
# evaluate using 10-fold cross validation
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
data, labels = load_train()
new_results = cross_val_score(model_transfer_kfold, data, labels, cv=kfold, scoring= 'accuracy')
print(new_results)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_18 (InputLayer)       [(None, 96, 96, 3)]       0         
                                                                 
 resnet50v2 (Functional)     (None, None, None, 2048)  23564800  
                                                                 
 tf.nn.relu_7 (TFOpLambda)   (None, 3, 3, 2048)        0         
                                                                 
 flatten_7 (Flatten)         (None, 18432)             0         
                                                                 
 dense_14 (Dense)            (None, 128)               2359424   
                                                                 
 activation_14 (Activation)  (None, 128)               0         
                          

Epoch 1/30
45/45 [==============================] - 16s 35ms/step - loss: 0.4314 - accuracy: 0.7981
Epoch 2/30
45/45 [==============================] - 2s 34ms/step - loss: 0.0728 - accuracy: 0.9778
Epoch 3/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0265 - accuracy: 0.9953
Epoch 4/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0139 - accuracy: 0.9986
Epoch 5/30
45/45 [==============================] - 2s 34ms/step - loss: 0.0075 - accuracy: 0.9997
Epoch 6/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0056 - accuracy: 0.9997
Epoch 7/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0046 - accuracy: 0.9997
Epoch 8/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0041 - accuracy: 0.9997
Epoch 9/30
45/45 [==============================] - 2s 34ms/step - loss: 0.0039 - accuracy: 1.0000
Epoch 10/30
45/45 [==============================] - 2s 34ms/step - loss: 0.0032 - accuracy: 1.0000
Epoch 11

Epoch 1/30
45/45 [==============================] - 6s 36ms/step - loss: 0.4467 - accuracy: 0.7981
Epoch 2/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0763 - accuracy: 0.9783
Epoch 3/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0227 - accuracy: 0.9972
Epoch 4/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0113 - accuracy: 0.9981
Epoch 5/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0101 - accuracy: 0.9986
Epoch 6/30
45/45 [==============================] - 2s 34ms/step - loss: 0.0073 - accuracy: 0.9992
Epoch 7/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0057 - accuracy: 0.9992
Epoch 8/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0050 - accuracy: 0.9994
Epoch 9/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0045 - accuracy: 0.9989
Epoch 10/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0034 - accuracy: 1.0000
Epoch 11/

Epoch 1/30
45/45 [==============================] - 6s 35ms/step - loss: 0.4952 - accuracy: 0.7703
Epoch 2/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0981 - accuracy: 0.9644
Epoch 3/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0295 - accuracy: 0.9933
Epoch 4/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0140 - accuracy: 0.9983
Epoch 5/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0087 - accuracy: 0.9997
Epoch 6/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0077 - accuracy: 0.9992
Epoch 7/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0048 - accuracy: 0.9997
Epoch 8/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0055 - accuracy: 0.9989
Epoch 9/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0056 - accuracy: 0.9992
Epoch 10/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0033 - accuracy: 1.0000
Epoch 11/

Epoch 1/30
45/45 [==============================] - 6s 36ms/step - loss: 0.4424 - accuracy: 0.7978
Epoch 2/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0840 - accuracy: 0.9692
Epoch 3/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0262 - accuracy: 0.9950
Epoch 4/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0171 - accuracy: 0.9969
Epoch 5/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0094 - accuracy: 0.9986
Epoch 6/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0079 - accuracy: 0.9989
Epoch 7/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0056 - accuracy: 0.9992
Epoch 8/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0059 - accuracy: 0.9992
Epoch 9/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0035 - accuracy: 1.0000
Epoch 10/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0029 - accuracy: 0.9997
Epoch 11/

...

Epoch 1/30
45/45 [==============================] - 6s 35ms/step - loss: 0.4363 - accuracy: 0.7919
Epoch 2/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0765 - accuracy: 0.9756
Epoch 3/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0246 - accuracy: 0.9956
Epoch 4/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0128 - accuracy: 0.9986
Epoch 5/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0066 - accuracy: 0.9997
Epoch 6/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0069 - accuracy: 0.9997
Epoch 7/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0045 - accuracy: 0.9997
Epoch 8/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0042 - accuracy: 1.0000
Epoch 9/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0050 - accuracy: 0.9994
Epoch 10/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0056 - accuracy: 0.9997
Epoch 11/

Epoch 1/30
45/45 [==============================] - 6s 35ms/step - loss: 0.5149 - accuracy: 0.7672
Epoch 2/30
45/45 [==============================] - 2s 35ms/step - loss: 0.1028 - accuracy: 0.9653
Epoch 3/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0272 - accuracy: 0.9956
Epoch 4/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0171 - accuracy: 0.9975
Epoch 5/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0111 - accuracy: 0.9975
Epoch 6/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0086 - accuracy: 0.9981
Epoch 7/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0062 - accuracy: 0.9997
Epoch 8/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0044 - accuracy: 0.9997
Epoch 9/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0064 - accuracy: 0.9997
Epoch 10/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0052 - accuracy: 0.9989
Epoch 11/

Epoch 1/30
45/45 [==============================] - 6s 35ms/step - loss: 0.4528 - accuracy: 0.7889
Epoch 2/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0804 - accuracy: 0.9728
Epoch 3/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0254 - accuracy: 0.9967
Epoch 4/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0128 - accuracy: 0.9986
Epoch 5/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0078 - accuracy: 0.9994
Epoch 6/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0069 - accuracy: 0.9986
Epoch 7/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0038 - accuracy: 1.0000
Epoch 8/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0058 - accuracy: 0.9992
Epoch 9/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0035 - accuracy: 1.0000
Epoch 10/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0032 - accuracy: 0.9997
Epoch 11/

Epoch 1/30
45/45 [==============================] - 6s 36ms/step - loss: 0.4505 - accuracy: 0.7839
Epoch 2/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0791 - accuracy: 0.9756
Epoch 3/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0245 - accuracy: 0.9944
Epoch 4/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0152 - accuracy: 0.9975
Epoch 5/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0082 - accuracy: 0.9992
Epoch 6/30
45/45 [==============================] - 2s 34ms/step - loss: 0.0065 - accuracy: 1.0000
Epoch 7/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0043 - accuracy: 1.0000
Epoch 8/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0039 - accuracy: 0.9997
Epoch 9/30
45/45 [==============================] - 2s 34ms/step - loss: 0.0043 - accuracy: 0.9997
Epoch 10/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0032 - accuracy: 1.0000
Epoch 11/

Epoch 1/30
45/45 [==============================] - 6s 36ms/step - loss: 0.4309 - accuracy: 0.7975
Epoch 2/30
45/45 [==============================] - 2s 34ms/step - loss: 0.0801 - accuracy: 0.9728
Epoch 3/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0234 - accuracy: 0.9964
Epoch 4/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0114 - accuracy: 0.9994
Epoch 5/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0071 - accuracy: 0.9997
Epoch 6/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0056 - accuracy: 0.9997
Epoch 7/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0055 - accuracy: 0.9994
Epoch 8/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0034 - accuracy: 0.9997
Epoch 9/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0064 - accuracy: 0.9992
Epoch 10/30
45/45 [==============================] - 2s 35ms/step - loss: 0.0036 - accuracy: 1.0000
Epoch 11/

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from sys import breakpointhook
#Normal Neural Net Architecture, Train Test split, image augmentation, Class Weightage, Test-Time Augmentation

# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
import scipy.io

import glob
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
#from pyimagesearch.smallervggnet import SmallerVGGNet
from keras.models import Sequential
from keras.layers import Dense, Conv2D, BatchNormalization, MaxPooling2D, Activation, Flatten, Dropout
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import argparse
import random
import pickle
import cv2
import os
#from tk import *
from sklearn.utils import compute_class_weight
#from tensorflow_docs.vis import embed
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import imageio
import re

batch_size = 80
num_channels = 3
num_classes = 1
image_size = 96
latent_dim = 100

generator_in_channels = latent_dim + num_classes
discriminator_in_channels = num_channels + num_classes
print(generator_in_channels, discriminator_in_channels)

discriminator = keras.Sequential(
    [
        keras.layers.InputLayer((96, 96, discriminator_in_channels)),
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.BatchNormalization(),
        #layers.GlobalMaxPooling2D(),
        layers.Dense(1),
    ],
    name="discriminator",
)

# Create the generator.
generator = keras.Sequential(
    [
        keras.layers.InputLayer((generator_in_channels,)),
        # We want to generate 128 + num_classes coefficients to reshape into a
        # 7x7x(128 + num_classes) map.
        layers.Dense(6 * 6 * generator_in_channels),
        layers.LeakyReLU(alpha=0.2),
        layers.BatchNormalization(),
        layers.Reshape((6, 6, generator_in_channels)),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"), #(80,12, 12, 128)
        layers.LeakyReLU(alpha=0.2),
        layers.BatchNormalization(),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"), #(80,24,24,128)
        layers.LeakyReLU(alpha=0.2),
        layers.BatchNormalization(),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"), #(80,48,48,128)
        layers.LeakyReLU(alpha=0.2),
        layers.BatchNormalization(),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"), #(80,96,96,128)
        layers.LeakyReLU(alpha=0.2),
        layers.BatchNormalization(),
        #layers.Conv2DTranspose(64, 20, 2, padding='valid', output_padding = (1,1)), #(80, 211,211,64)
        #layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(3, (7, 7), padding="same", activation="tanh"), #(80, 96,96,3)
        layers.LeakyReLU(alpha=0.2),
    ],
    name="generator",
)

print(discriminator.summary())
print(generator.summary())

#all_digits, all_labels = load_train()
#all_digits = np.reshape(all_digits, (-1, 96, 96, 3))
##all_labels = keras.utils.to_categorical(all_labels, 2)
##lb = LabelBinarizer()
##all_labels = lb.fit_transform(all_labels)

dataset = tf.data.Dataset.from_tensor_slices((all_digits, all_labels))
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)

print(f"Shape of training images: {all_digits.shape}")
print(f"Shape of training labels: {all_labels.shape}")



class ConditionalGAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super(ConditionalGAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.gen_loss_tracker = keras.metrics.Mean(name="generator_loss")
        self.disc_loss_tracker = keras.metrics.Mean(name="discriminator_loss")

    @property
    def metrics(self):
        return [self.gen_loss_tracker, self.disc_loss_tracker]

    def compile(self, d_optimizer, g_optimizer, g_loss_fn, d_loss_fn):
        super(ConditionalGAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.g_loss_fn = g_loss_fn
        self.d_loss_fn = d_loss_fn

    def gradient_penalty(self, batch_size, real_images, fake_images):
        """Calculates the gradient penalty.

        This loss is calculated on an interpolated image
        and added to the discriminator loss.
        """
        # Get the interpolated image
        alpha = tf.random.normal([batch_size, 1, 1, 1], 0.0, 1.0)
        diff = fake_images - real_images
        interpolated = real_images + alpha * diff

        with tf.GradientTape() as gp_tape:
            gp_tape.watch(interpolated)
            # 1. Get the discriminator output for this interpolated image.
            pred = self.discriminator(interpolated)

        # 2. Calculate the gradients w.r.t to this interpolated image.
        grads = gp_tape.gradient(pred, [interpolated])[0]
        # 3. Calculate the norm of the gradients.
        norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=[1, 2, 3]))
        gp = tf.reduce_mean((norm - 1.0) ** 2)
        return gp
    def train_step(self, data):
        # Unpack the data.
        real_images, one_hot_labels = data
        one_hot_labels = tf.cast(one_hot_labels, tf.float32)
        real_images = tf.cast(real_images, tf.float32)

        # Add dummy dimensions to the labels so that they can be concatenated with
        # the images. This is for the discriminator.
        image_one_hot_labels = one_hot_labels[:, :, None, None]
        image_one_hot_labels = tf.repeat(
            image_one_hot_labels, repeats=[image_size * image_size]
        )
        image_one_hot_labels = tf.reshape(
            image_one_hot_labels, (-1, image_size, image_size, num_classes)
        )

        # Sample random points in the latent space and concatenate the labels.
        # This is for the generator.
        batch_size = tf.shape(real_images)[0]
        for i in range(5):
          random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
          print(one_hot_labels)
          #print(random_latent_vectors.shape)
          #Concatenates labels with generator input
          random_vector_labels = tf.concat([random_latent_vectors, one_hot_labels], axis=1)

        # Decode the noise (guided by labels) to fake images.
          with tf.GradientTape() as tape:
            generated_images = self.generator(random_vector_labels)

        # Combine them with real images. Note that we are concatenating the labels
        # with these images here.
            #concatenates labels with discriminator input
            fake_image_and_labels = tf.concat([generated_images, image_one_hot_labels], -1)
            real_image_and_labels = tf.concat([real_images, image_one_hot_labels], -1)
        #combined_images = tf.concat(
        #    [fake_image_and_labels, real_image_and_labels], axis=0
        #)

        # Assemble labels discriminating real from fake images.
        #labels = tf.concat(
        #    [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        #)

        # Train the discriminator.
          
            fake_logits = self.discriminator(fake_image_and_labels) #used to be combined_images in parameter
            real_logits = self.discriminator(real_image_and_labels)

            #d_loss = self.loss_fn_d(labels, predictions)
            d_cost = self.d_loss_fn(real_img=real_logits, fake_img=fake_logits)
                # Calculate the gradient penalty
            gp = self.gradient_penalty(batch_size, real_image_and_labels, fake_image_and_labels)
                # Add the gradient penalty to the original discriminator loss
            d_loss = d_cost + gp * 10.0
          grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
          self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        # Sample random points in the latent space.
        random_latent_vectors1 = tf.random.normal(shape=(batch_size, self.latent_dim))
        random_vector_labels1 = tf.concat([random_latent_vectors1, one_hot_labels], axis=1)

        # Assemble labels that say "all real images".
        #misleading_labels = tf.zeros((batch_size, 1))

        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
          fake_images1 = self.generator(random_vector_labels1)
          fake_image_and_labels1 = tf.concat([fake_images1, image_one_hot_labels], -1)
          predictions = self.discriminator(fake_image_and_labels1)
          g_loss = self.g_loss_fn(predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # Monitor loss.
        #self.gen_loss_tracker.update_state(g_loss)
        #self.disc_loss_tracker.update_state(d_loss)
        #return {
        #    "g_loss": self.gen_loss_tracker.result(),
        #    "d_loss": self.disc_loss_tracker.result(),
        #}
        return {"d_loss": d_loss, "g_loss": g_loss}
cond_gan = ConditionalGAN(
    discriminator=discriminator, generator=generator, latent_dim=latent_dim
)
def discriminator_loss(real_img, fake_img):
    real_loss = tf.reduce_mean(real_img)
    fake_loss = tf.reduce_mean(fake_img)
    return fake_loss - real_loss


# Define the loss functions for the generator.
def generator_loss(fake_img):
    return -tf.reduce_mean(fake_img)
cond_gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=0.0002, beta_1 = 0.5),
    g_optimizer=keras.optimizers.Adam(learning_rate=0.0002, beta_1 = 0.5),
    d_loss_fn=discriminator_loss, g_loss_fn = generator_loss
)

cond_gan.fit(dataset, epochs=50)

101 4
Model: "discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 48, 48, 64)        2368      
                                                                 
 leaky_re_lu_8 (LeakyReLU)   (None, 48, 48, 64)        0         
                                                                 
 batch_normalization_7 (Batc  (None, 48, 48, 64)       256       
 hNormalization)                                                 
                                                                 
 conv2d_4 (Conv2D)           (None, 24, 24, 128)       73856     
                                                                 
 leaky_re_lu_9 (LeakyReLU)   (None, 24, 24, 128)       0         
                                                                 
 batch_normalization_8 (Batc  (None, 24, 24, 128)      512       
 hNormalization)                               

NameError: ignored

In [ ]:
import matplotlib.pyplot as plt
import numpy
trained_gen = cond_gan.generator

# Choose the number of intermediate images that would be generated in
# between the interpolation + 2 (start and last images).
num_interpolation = 2000  # param {type:"integer"}

# Sample noise for the interpolation.
interpolation_noise = tf.random.normal(shape=(1, latent_dim))
interpolation_noise = tf.repeat(interpolation_noise, repeats=num_interpolation)
interpolation_noise = tf.reshape(interpolation_noise, (num_interpolation, latent_dim))


def interpolate_class_1_2():
    # Convert the start and end labels to one-hot encoded vectors.
    #first_label = keras.utils.to_categorical([first_number], num_classes)
    #second_label = keras.utils.to_categorical([second_number], num_classes)
    #first_label = tf.cast(first_label, tf.float32)
    #second_label = tf.cast(second_label, tf.float32)

    # Calculate the interpolation vector between the two labels.
    #percent_second_label = tf.linspace(0, 1, num_interpolation)[:, None]
    #percent_second_label = tf.cast(percent_second_label, tf.float32)
    interpolation_labels = np.zeros((2000, 1), dtype=np.float32)

    # Combine the noise and the labels and run inference with the generator.
    noise_and_labels = tf.concat([interpolation_noise, interpolation_labels], 1)
    fake = trained_gen.predict(noise_and_labels)
    return fake


start_class = 1  # param {type:"slider", min:0, max:9, step:1}
end_class = 5  # param {type:"slider", min:0, max:9, step:1}

fake_images = interpolate_class_1_2()
fake_images_disp = fake_images * 255.0
fake_images_disp = fake_images_disp.astype(np.uint8)
#print(fake_images_disp[0].shape)

#plt.imshow(fake_images_disp[1])
#plt.imshow(fake_images_disp[2])
#plt.imshow(fake_images_disp[3])
#plt.imshow(fake_images_disp[4])
plt.imshow(fake_images_disp[0])
for i in range(2000):
  #mdic = {"Generated Image " + str(i+1): fake_images[i]}
  plt.imsave("/content/drive/MyDrive/Wilson/cGAN Dataset by Histo Score - 24 hrs/Histo Score 1-2/Generated Image " + str(i+1) + ".png", fake_images[i])

def interpolate_class_3_4():
    interpolation_labels = np.ones((2000, 1), dtype=np.float32)

    # Combine the noise and the labels and run inference with the generator.
    noise_and_labels = tf.concat([interpolation_noise, interpolation_labels], 1)
    fake = trained_gen.predict(noise_and_labels)
    return fake
fake_images_2 = interpolate_class_3_4()
fake_images_2_disp = fake_images_2 * 255.0
fake_images_2_disp = fake_images_2_disp.astype(np.uint8)
plt.figure()
plt.imshow(fake_images_2_disp[0])
#plt.imshow(fake_images_2_disp[1])
#plt.imshow(fake_images_2_disp[2])
#plt.imshow(fake_images_2_disp[3])
#plt.imshow(fake_images_2_disp[4])
plt.show()
for i in range(2000):
  #mdic = {"Generated Image " + str(i+2001): fake_images_2[i]}
  plt.imsave("/content/drive/MyDrive/Wilson/cGAN Dataset by Histo Score - 24 hrs/Histo Score 3-4/Generated Image " + str(i+2001) + ".png", fake_images_2[i])

63/63 [==============================] - 1s 14ms/step
